
### Section 3.1 – Preparing Z-Score Rasters

This section involves generating **Z-score-based flood probability rasters** from DEM and headwater height inputs. These rasters help quantify the likelihood of inundation while accounting for model and elevation uncertainty.

---

#### 📁 Set Up Project Directory

Ensure your folder structure is consistent with earlier stages of the project. In the script, update the `project_dir` variable to match the root directory where your input rasters and output folders are located:

```python
project_dir = r"X:\path\to\your\project"
```

All resulting Z-score rasters and mosaics will be saved under this directory.

---

#### 📄 Prepare the Excel Input File

The script expects an Excel input file named:

```
cascade_inputs_revised.xlsx
```

This file should include the following columns:

| Column Name | Description |
|-------------|-------------|
| `HUCs`      | Sub-basin name or identifier |
| `HUC_ID`    | Hydrologic Unit Code (e.g., 03090206) |
| `Z_values`  | Headwater height (in ft) from the hydrologic model |
| `Scenario`  | Scenario label (e.g., "r1d100yr", "king_tide_2.6ft") |

Each row should correspond to a single flood scenario for a specific HUC boundary.

---

#### ⚙️ What the Script Does

For each row in the Excel file:

1. Loads the `dem_ft.tif` file corresponding to the `HUC_ID`.
2. Computes the Z-score raster using the formula:

   ```
   Z = (Z_value - DEM) / Uncertainty
   ```

   where Uncertainty is typically a fixed value (e.g., 0.46 ft).
   
3. Converts the Z-score into a probability raster.
4. Saves the raster with a scenario-specific name inside the HUC directory.

---

#### 🗂 Output

- Individual Z-score rasters saved under each HUC folder
- Scenario-level mosaics saved in:

```
project_dir/Zscores/Mosaic/
```

Mosaics are grouped by scenario, combining all Z-scores from different HUCs for that event.

---

#### ✅ Final Notes

- Confirm that the DEM rasters and HUC folder names match the entries in your Excel file.
- Ensure the input Excel file is formatted correctly with no missing or invalid entries.
- Z-scores help represent probabilistic flood risk rather than binary flood/no-flood results, improving decision-making for infrastructure planning.



In [7]:
%%time
import os
import glob
import pandas as pd
from arcpy.sa import *
from arcpy.management import DefineProjection

arcpy.env.addOutputsToMap = False
arcpy.env.overwriteOutput = True
arcpy.CheckOutExtension("Spatial")

# Define the project directory
project_dir = r'X:\amandal2023$\Working\Homestead\Homestead_City'
huc12s_dir = os.path.join(project_dir, 'HUC12s')
cascade_inputs_path = os.path.join(project_dir, 'Cascade', 'Inputs', 'cascade_inputs_revised.xlsx')
mosaic_dir = os.path.join(project_dir, 'mosaic')
Huc_col = 'HUC_ID'

# Read the Cascade inputs Excel file
df = pd.read_excel(cascade_inputs_path)
df[Huc_col] = df[Huc_col].astype(str).str.zfill(12)  # Ensure HUC IDs are strings with leading zeros
scenarios = df['Scenario'].unique()

# Use the glob library to search for .tif files in the HUC12s directory
dem_files = glob.glob(os.path.join(huc12s_dir, '**', 'dem_ft.tif'), recursive=True)

def create_output_folder(output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

mlist = []  # Initialize the list for storing output rasters

# Process each DEM file
for dem_file in dem_files:
    # Extract HUC ID and name from the directory structure
    directory, filename = os.path.split(dem_file)
    hucid_name = os.path.basename(directory)
    hucid = hucid_name.split('_')[0].zfill(12)  # Ensure HUC ID has leading zeros
    hucname = ' '.join(hucid_name.split('_')[1:]).title()

    # Print extracted HUC ID and name
    print(f'Processing HUC ID: {hucid}, HUC Name: {hucname}')

    # Get the spatial reference of the DEM file
    dem_sr = arcpy.Describe(dem_file).spatialReference

    for scenario in scenarios:
        df_scenario = df.loc[df['Scenario'] == scenario]
        matching_rows = df_scenario.loc[df_scenario[Huc_col] == hucid]

        # Print diagnostic information
        print(f'Checking scenario: {scenario}, HUC ID: {hucid}')
        print(f'Matching rows for scenario {scenario}: {matching_rows}')

        if not matching_rows.empty:
            z_value = matching_rows['Z_values'].values[0]

            # Define the output folder and file paths
            #output_folder = os.path.join(directory, 'Inundation', str(scenario))
            #create_output_folder(output_folder)

            #out_file = filename.replace('dem_ft', scenario)
            #output_raster = os.path.join(output_folder, out_file)
            output_raster = dem_file.replace('dem_ft', 'Inundation_'+scenario)

            # Calculate the Z-score raster
            outras = RasterCalculator([z_value, dem_file, 0.46], ["x", "y", "z"], "float((x - y) / z)")
            outras.save(output_raster)

            # Ensure the output raster has the same spatial reference as the DEM file
            DefineProjection(output_raster, dem_sr)

            # Append the output raster to the list for mosaicking
            mlist.append(output_raster)

            print(f'Processed {output_raster} with Z-value: {z_value}')
        else:
            # Print debug information about why no match was found
            print(f"No matching Z-value found for HUC ID {hucid} and scenario {scenario}")
            print(f"Available HUC IDs in the DataFrame for scenario {scenario}: {df_scenario[Huc_col].unique()}")

# Create mosaics for each scenario
for scenario in scenarios:
    scenario_rasters = [r for r in mlist if scenario in r]
    if scenario_rasters:
        mosaic_name = f'Inundation_{scenario}.tif'
        arcpy.MosaicToNewRaster_management(scenario_rasters, mosaic_dir, mosaic_name, dem_sr, "32_BIT_FLOAT", "", "1", "LAST", "FIRST")

print("Processing completed.")


Processing HUC ID: 030902061605, HUC Name: Mowerycanal
Checking scenario: 1d5yr, HUC ID: 030902061605
Matching rows for scenario 1d5yr:          HUC_ID          HUCs  Z_values Scenario
0  030902061605  MOWERY CANAL      2.82    1d5yr
Processed X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061605_MoweryCanal\Inundation_1d5yr.tif with Z-value: 2.82
Checking scenario: 1d10yr, HUC ID: 030902061605
Matching rows for scenario 1d10yr:          HUC_ID          HUCs  Z_values Scenario
3  030902061605  MOWERY CANAL      3.25   1d10yr
Processed X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061605_MoweryCanal\Inundation_1d10yr.tif with Z-value: 3.25
Checking scenario: 1d100yr, HUC ID: 030902061605
Matching rows for scenario 1d100yr:          HUC_ID          HUCs  Z_values Scenario
6  030902061605  MOWERY CANAL      4.96  1d100yr
Processed X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061605_MoweryCanal\Inundation_1d100yr.tif with Z-value: 4.96
Checki

72  030902061605  MOWERY CANAL      5.34  1d100yr_1ft
Processed X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061605_MoweryCanal\Inundation_1d100yr_1ft.tif with Z-value: 5.34
Checking scenario: 1d100yr_2ft, HUC ID: 030902061605
Matching rows for scenario 1d100yr_2ft:           HUC_ID          HUCs  Z_values     Scenario
75  030902061605  MOWERY CANAL      5.87  1d100yr_2ft
Processed X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061605_MoweryCanal\Inundation_1d100yr_2ft.tif with Z-value: 5.87
Checking scenario: 1d100yr_3ft, HUC ID: 030902061605
Matching rows for scenario 1d100yr_3ft:           HUC_ID          HUCs  Z_values     Scenario
78  030902061605  MOWERY CANAL      6.37  1d100yr_3ft
Processed X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061605_MoweryCanal\Inundation_1d100yr_3ft.tif with Z-value: 6.37
Checking scenario: 1d100yr_4ft, HUC ID: 030902061605
Matching rows for scenario 1d100yr_4ft:           HUC_ID          HUCs  Z_values

73  030902061606  CANAL S177       5.8  1d100yr_1ft
Processed X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061606_CanalS177\Inundation_1d100yr_1ft.tif with Z-value: 5.8
Checking scenario: 1d100yr_2ft, HUC ID: 030902061606
Matching rows for scenario 1d100yr_2ft:           HUC_ID        HUCs  Z_values     Scenario
76  030902061606  CANAL S177      6.03  1d100yr_2ft
Processed X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061606_CanalS177\Inundation_1d100yr_2ft.tif with Z-value: 6.03
Checking scenario: 1d100yr_3ft, HUC ID: 030902061606
Matching rows for scenario 1d100yr_3ft:           HUC_ID        HUCs  Z_values     Scenario
79  030902061606  CANAL S177      6.36  1d100yr_3ft
Processed X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061606_CanalS177\Inundation_1d100yr_3ft.tif with Z-value: 6.36
Checking scenario: 1d100yr_4ft, HUC ID: 030902061606
Matching rows for scenario 1d100yr_4ft:           HUC_ID        HUCs  Z_values     Scenario
82  0

71  030902061607  MODEL LAND CANAL      9.15  1d10yr_5ft_kt
Processed X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061607_ModelLandCanal\Inundation_1d10yr_5ft_kt.tif with Z-value: 9.15
Checking scenario: 1d100yr_1ft, HUC ID: 030902061607
Matching rows for scenario 1d100yr_1ft:           HUC_ID              HUCs  Z_values     Scenario
74  030902061607  MODEL LAND CANAL      3.21  1d100yr_1ft
Processed X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061607_ModelLandCanal\Inundation_1d100yr_1ft.tif with Z-value: 3.21
Checking scenario: 1d100yr_2ft, HUC ID: 030902061607
Matching rows for scenario 1d100yr_2ft:           HUC_ID              HUCs  Z_values     Scenario
77  030902061607  MODEL LAND CANAL      4.15  1d100yr_2ft
Processed X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061607_ModelLandCanal\Inundation_1d100yr_2ft.tif with Z-value: 4.15
Checking scenario: 1d100yr_3ft, HUC ID: 030902061607
Matching rows for scenario 1d100yr_3ft:        


### Section 3.2 – Calculation of Composite Score for Critical Facilities

This section guides the calculation of a composite flood risk score for critical facilities based on their exposure to modeled flood scenarios and their assigned consequence tiers. The outputs help prioritize infrastructure improvements and inform resilience strategies.

---

#### 📌 Purpose

The script performs a spatial risk assessment that includes:
- Inundation area analysis
- Assignment of flood risk and consequence factors
- Calculation of a weighted composite score for each facility

These metrics are crucial for identifying and ranking high-risk assets within a watershed or jurisdiction.

---

#### 🧭 Step-by-Step Workflow

1. **Addition of UID Field**

A `UID` field is added to the input feature class to uniquely identify each critical facility. This ensures that all analysis results can be accurately traced back to each record.

2. **Coordinate Reference System (CRS) Alignment**

The script checks whether the CRS of the feature class and the flood inundation raster match. If needed, the raster is reprojected to align with the feature layer to ensure accurate area-based analysis.

3. **Inundation Percentage Calculation**

Each facility is intersected with the flood raster or flood polygon for a given scenario. The percentage of the facility’s area that is inundated is calculated as:

```
Inundation % = (Inundated Area / Total Facility Area) × 100
```

4. **Flood Risk Factor Assignment**

The flood risk factor is categorized based on the following thresholds:

| Inundation % Range       | Flood Risk Factor |
|--------------------------|-------------------|
| ≥ 90%                    | 6                 |
| 80% to < 90%             | 5                 |
| 70% to < 80%             | 4                 |
| 60% to < 70%             | 3                 |
| 50% to < 60%             | 2                 |
| < 50%                    | 1                 |

This value reflects the severity of flood exposure for each facility.

5. **Consequence Risk Factor**

The consequence score is based on a predefined tiering system, where lower tiers indicate higher vulnerability. The consequence factor is calculated as:

```
Consequence Risk Factor = 7 − Tier
```

For example, a Tier 1 facility (most critical) would get a consequence score of 6.

6. **Composite Score Calculation**

The final risk score is a weighted combination of both flood and consequence risk factors:

```
Composite Score = (Flood Risk Factor × 0.25) + (Consequence Risk Factor × 0.75)
```

This formula places higher weight on the consequence of failure, ensuring that more critical facilities are ranked higher even under moderate flood conditions.

---

#### 📤 Output

The script outputs a feature class or table with the following fields:

- UID
- Inundation Percentage
- Flood Risk Factor
- Consequence Risk Factor
- Composite Score

These outputs can be visualized using graduated color maps or used in prioritization dashboards.

---

#### ✅ Final Notes

- Ensure input rasters are properly thresholded and converted to polygons if needed.
- The tier value must be a numeric field in the input feature class.
- This script assumes facilities are polygon features and not point-based.



In [4]:
%%time
import arcpy
import os
from arcpy.sa import *

arcpy.env.addOutputsToMap = False
arcpy.env.overwriteOutput = True
arcpy.CheckOutExtension("Spatial")

# Define the base directory and geodatabase
basedir = r'X:\amandal2023$\Working\Homestead\Homestead_City'
gdb = os.path.join(basedir, 'Homestead.gdb')
tiers_fc = os.path.join(gdb, 'critical_facilities')
mosaic_dir = os.path.join(basedir, 'mosaic')
temp_dir = os.path.join(basedir, 'Temp')  # Temporary directory for storing intermediate rasters
scenarios = ['1d100yr']  # List of scenarios to process

# Create temp directory if it does not exist
if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)

# Function to add uid field and assign unique IDs
def add_uid_field(tiers_fc):
    if 'uid' not in [f.name for f in arcpy.ListFields(tiers_fc)]:
        arcpy.AddField_management(tiers_fc, 'uid', 'LONG')
        with arcpy.da.UpdateCursor(tiers_fc, 'uid') as cursor:
            uid = 1
            for row in cursor:
                row[0] = uid
                cursor.updateRow(row)
                uid += 1

# Function to ensure CRS match and project if needed
def ensure_crs_match(input_raster, target_fc):
    raster_sr = arcpy.Describe(input_raster).spatialReference
    target_sr = arcpy.Describe(target_fc).spatialReference
    if raster_sr != target_sr:
        projected_raster = os.path.join(temp_dir, f'projected_{os.path.basename(input_raster)}')
        arcpy.ProjectRaster_management(input_raster, projected_raster, target_sr)
        return projected_raster
    return input_raster

# Function to calculate inundation percentage
def calculate_inundation_percentage(tiers_fc, mosaic_raster, scenario):
    try:
        # Ensure the CRS of the inundation raster matches the CRS of the tiers feature class
        inundation_raster = ensure_crs_match(mosaic_raster, tiers_fc)
        
        # Apply Con to extract values > 0
        inundation_raster = Con(Raster(inundation_raster) > 0, 1)
        
        # Save the temporary raster in the temp directory
        temp_inundation_raster = os.path.join(temp_dir, f'inundation_{scenario}.tif')
        inundation_raster.save(temp_inundation_raster)
        
        # Convert the inundation raster to a polygon
        inundation_polygon = os.path.join(temp_dir, f'inundation_{scenario}.shp')
        arcpy.RasterToPolygon_conversion(temp_inundation_raster, inundation_polygon, "NO_SIMPLIFY", "Value")
        
        # Ensure uid field exists in the polygons
        if 'uid' not in [f.name for f in arcpy.ListFields(inundation_polygon)]:
            arcpy.AddField_management(inundation_polygon, 'uid', 'LONG')
            arcpy.CalculateField_management(inundation_polygon, 'uid', '!gridcode!', "PYTHON3")
        
        # Calculate the intersection area using TabulateIntersection
        zonal_table = os.path.join(gdb, f'zonal_table_{scenario}')
        if arcpy.Exists(zonal_table):
            arcpy.Delete_management(zonal_table)
        
        arcpy.analysis.TabulateIntersection(
            in_zone_features=tiers_fc,
            zone_fields="uid",
            in_class_features=inundation_polygon,
            out_table=zonal_table,
            class_fields=None,
            sum_fields=None,
            xy_tolerance=None,
            out_units="SQUARE_METERS")
        
        # Join the zonal table to the tiers feature class using JoinField
        join_fields = ["Percentage"]  # Specify the fields you want to join from the zonal table
        arcpy.management.JoinField(tiers_fc, "uid", zonal_table, "uid", join_fields)
        
        # Save the result as a new feature class
        output_fc = os.path.join(gdb, f'com_score_{scenario}')
        
        # Delete output_fc if it exists
        if arcpy.Exists(output_fc):
            arcpy.Delete_management(output_fc)
        
        arcpy.CopyFeatures_management(tiers_fc, output_fc)
        print(f'Saved: {output_fc}')
        
        # Fill NULL values in the Percentage column with zero
        arcpy.MakeFeatureLayer_management(output_fc, "output_layer")
        arcpy.SelectLayerByAttribute_management("output_layer", "NEW_SELECTION", "Percentage IS NULL")
        arcpy.CalculateField_management("output_layer", "Percentage", 0, "PYTHON3")
        
        # Calculate Flood Risk Factor based on the Percentage column in the output_fc
        calculate_flood_risk_factor(output_fc)
        
        # Add Consequence Risk Factor and Composite Score fields
        add_risk_and_composite_scores(output_fc)
        
        # Delete the temporary zonal table
        if arcpy.Exists(zonal_table):
            arcpy.Delete_management(zonal_table)

        # Delete the Percentage field from tiers_fc
        if 'Percentage' in [f.name for f in arcpy.ListFields(tiers_fc)]:
            arcpy.DeleteField_management(tiers_fc, 'Percentage')
    except Exception as e:
        print(f"An error occurred for scenario {scenario}: {e}")

# Function to calculate flood risk factor using select by attribute
def calculate_flood_risk_factor(fc):
    if 'Flood_Risk_Factor' not in [f.name for f in arcpy.ListFields(fc)]:
        arcpy.AddField_management(fc, "Flood_Risk_Factor", "SHORT")
    
    arcpy.MakeFeatureLayer_management(fc, "temp_layer")
    
    # Define flood risk factor categories
    categories = [
        ("{}.Percentage >= 90".format(fc), 6),
        ("{}.Percentage >= 80 AND {}.Percentage < 90".format(fc, fc), 5),
        ("{}.Percentage >= 70 AND {}.Percentage < 80".format(fc, fc), 4),
        ("{}.Percentage >= 60 AND {}.Percentage < 70".format(fc, fc), 3),
        ("{}.Percentage >= 50 AND {}.Percentage < 60".format(fc, fc), 2),
        ("{}.Percentage < 50".format(fc), 1)
    ]
    
    # Update Flood_Risk_Factor based on the defined categories
    for where_clause, risk_factor in categories:
        arcpy.SelectLayerByAttribute_management("temp_layer", "NEW_SELECTION", where_clause)
        arcpy.CalculateField_management("temp_layer", "Flood_Risk_Factor", risk_factor, "PYTHON3")
    
    arcpy.Delete_management("temp_layer")

# Function to add Consequence Risk Factor and Composite Score fields
def add_risk_and_composite_scores(fc):
    if 'Consequence_Risk_Factor' not in [f.name for f in arcpy.ListFields(fc)]:
        arcpy.AddField_management(fc, "Consequence_Risk_Factor", "SHORT")
    
    if 'Composite_Score' not in [f.name for f in arcpy.ListFields(fc)]:
        arcpy.AddField_management(fc, "Composite_Score", "DOUBLE")
    
    arcpy.MakeFeatureLayer_management(fc, "temp_layer")

    # Calculate Consequence Risk Factor
    arcpy.CalculateField_management("temp_layer", "Consequence_Risk_Factor", "7 - !tiers!", "PYTHON3")

    # Calculate Composite Score
    arcpy.CalculateField_management("temp_layer", "Composite_Score", "(!Flood_Risk_Factor! * 0.25) + (!Consequence_Risk_Factor! * 0.75)", "PYTHON3")
    
    arcpy.Delete_management("temp_layer")

# Add uid field and assign unique IDs to tiers
add_uid_field(tiers_fc)

# Process each scenario
for scenario in scenarios:
    mosaic_raster = os.path.join(mosaic_dir, f'inundation_{scenario}.tif')
    if arcpy.Exists(mosaic_raster):
        calculate_inundation_percentage(tiers_fc, mosaic_raster, scenario)
    else:
        print(f'Mosaic raster for scenario {scenario} not found: {mosaic_raster}')

print("Processing completed.")


Saved: X:\amandal2023$\Working\Homestead\Homestead_City\Homestead.gdb\com_score_1d100yr
Processing completed.
Wall time: 2min 18s
